# Fill Missing Frames for 'not detect' Gaps in Trajectory Data

This notebook provides a template for filling missing frame data in your vehicle trajectory datasets.  
It was designed to work with the missing frames information contained in `missing_frames_detail.csv` and multiple trajectory files (in CSV/XLSX format) that have been split into parts due to their large size.  

The goal is to fill in gaps where the `missing reasons` column is **`not detect`** and return a completed trajectory dataset for each split file.

**Key features of the notebook:**

- **Configuration section**: specify the path to the `missing_frames_detail.csv` file and list the trajectory files you wish to process.  
- **Gap filtering**: load missing gap information and filter to only those gaps where the `missing reasons` column is `'not detect'`.  
- **Interpolation logic**: for each missing gap, find the nearest existing frames for the same vehicle before and after the gap, and linearly interpolate numeric fields (e.g. `x`, `y`, `w`, `h`, `s_m`, `vehicle_length`, `vehicle_width`) across missing frames.  
  - Discrete/categorical fields (e.g. `class_id`, `vehicle_id`, `lane_observed`, `lane_smoothed`) are copied from the preceding frame when available.  
  - `confidence` is set to `0.0` for all filled rows.  
  - `is_lane_change` is set to `0` in filled sections to avoid falsely introducing lane changes.  
- **Time calculations**: compute `time_s` as `frame_id / FPS` and derive `time_min_sec` via Python's `timedelta` formatting.  
- **Audit columns**: optional columns (e.g. `fill_flag`, `fill_method`, `fill_lane_conflict`) can be added to mark rows that were created during filling.  
- **Output**: write the filled dataset for each input file as both CSV and XLSX files, preserving the original column order.  

> **Important:**
>
> - **Performance considerations**: because trajectory data can be extremely large, this template reads each file one at a time and operates on that subset.  
> - You may need to adapt the interpolation logic to fit your specific schema; double-check your column names and the meaning of each field.  
> - The paths in the configuration section are placeholders; update them to match your local environment before running the notebook.


In [5]:
# Configuration
import os
import pandas as pd
import numpy as np
from datetime import timedelta

# Path to missing frames detail CSV (update as needed)
MISSING_FRAMES_PATH = '/Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/missing_frames_detail.csv'

# List of trajectory files to process (update to your local file paths)
TRAJ_FILES = [
    r'/Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part1.xlsx',
    r'/Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part2.xlsx',
    r'/Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part3.xlsx',
    r'/Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part4.xlsx',
]

# Frames per second of the video
FPS = 59.94

# Pixel to metre conversion factor (1 pixel = 0.102 m)
PIXEL_TO_METRE = 0.102

# Output directory for filled files
OUTPUT_DIR = 'filled_outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [6]:
def format_time_min_sec(seconds: float) -> str:
    '''Convert seconds to a HH:MM:SS string.'''
    td = timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    secs = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"


def interpolate_numeric(start_val, end_val, start_frame, end_frame, target_frame):
    '''Linearly interpolate a value between two frames.
    If one boundary is missing, return the available boundary value.'''
    if start_val is None:
        return end_val
    if end_val is None:
        return start_val
    if start_frame == end_frame:
        return start_val
    fraction = (target_frame - start_frame) / (end_frame - start_frame)
    return start_val + fraction * (end_val - start_val)


In [7]:
def fill_missing_gaps(df: pd.DataFrame, missing_info: pd.DataFrame, fps: float = FPS) -> pd.DataFrame:
    '''Fill missing frames for a single trajectory DataFrame based on gap information.'''
    output_df = df.copy()

    # Identify numeric columns that may need interpolation
    numeric_cols = ['x', 'y', 'w', 'h', 'vehicle_length', 'vehicle_width', 'x_m', 'y_m', 's_m']
    numeric_cols = [col for col in numeric_cols if col in output_df.columns]

    # Identify categorical columns to propagate
    categorical_cols = [col for col in ['class_id', 'vehicle_id', 'lane_observed', 'lane_smoothed',
                                        'lane_change_from', 'lane_change_to', 'combine veh uids'] if col in output_df.columns]

    for _, gap in missing_info.iterrows():
        veh_uid = gap['vehicle_uid']
        start_frame = int(gap['gap_start_frame'])
        end_frame = int(gap['gap_end_frame'])

        veh_df = output_df[output_df['vehicle_uid'] == veh_uid]
        if veh_df.empty:
            continue

        prev_row = veh_df[veh_df['frame_id'] < start_frame].sort_values('frame_id').tail(1)
        next_row = veh_df[veh_df['frame_id'] > end_frame].sort_values('frame_id').head(1)
        if prev_row.empty and next_row.empty:
            continue

        prev_frame = prev_row['frame_id'].iloc[0] if not prev_row.empty else None
        next_frame = next_row['frame_id'].iloc[0] if not next_row.empty else None
        prev_vals = prev_row.iloc[0] if not prev_row.empty else None
        next_vals = next_row.iloc[0] if not next_row.empty else None

        new_rows = []
        for frame in range(start_frame, end_frame + 1):
            if ((output_df['vehicle_uid'] == veh_uid) & (output_df['frame_id'] == frame)).any():
                continue
            row_dict = {
                'frame_id': frame,
                'vehicle_uid': veh_uid
            }
            # interpolate numeric fields
            for col in numeric_cols:
                prev_val = prev_vals[col] if prev_vals is not None and col in prev_vals else None
                next_val = next_vals[col] if next_vals is not None and col in next_vals else None
                row_dict[col] = interpolate_numeric(prev_val, next_val, prev_frame, next_frame, frame)
            # convert to metres if x,y exist
            if 'x_m' in numeric_cols and 'x' in output_df.columns:
                row_dict['x_m'] = row_dict['x'] * PIXEL_TO_METRE
            if 'y_m' in numeric_cols and 'y' in output_df.columns:
                row_dict['y_m'] = row_dict['y'] * PIXEL_TO_METRE
            # propagate categorical fields
            for col in categorical_cols:
                if prev_vals is not None and col in prev_vals:
                    row_dict[col] = prev_vals[col]
                elif next_vals is not None and col in next_vals:
                    row_dict[col] = next_vals[col]
            # set confidence and lane change flags
            if 'confidence' in output_df.columns:
                row_dict['confidence'] = 0.0
            if 'is_lane_change' in output_df.columns:
                row_dict['is_lane_change'] = 0
            # compute time
            if 'time_s' in output_df.columns:
                row_dict['time_s'] = frame / fps
            if 'time_min_sec' in output_df.columns:
                row_dict['time_min_sec'] = format_time_min_sec(row_dict['time_s'])
            new_rows.append(row_dict)
        if new_rows:
            new_df = pd.DataFrame(new_rows)
            output_df = pd.concat([output_df, new_df], ignore_index=True)
    output_df = output_df.sort_values(['vehicle_uid', 'frame_id']).reset_index(drop=True)
    return output_df


In [8]:
# Load missing frames detail and filter to 'not detect' reasons
missing_df = pd.read_csv(MISSING_FRAMES_PATH)
not_detect_gaps = missing_df[missing_df['missing reasons'].str.lower() == 'not detect']

# Prepare log
log_entries = []

for file_path in TRAJ_FILES:
    print(f"Processing {file_path}")
    # load data
    if file_path.lower().endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.lower().endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path)
    else:
        print(f"Unsupported file type: {file_path}")
        continue
    original_count = len(df)
    # fill gaps
    filled_df = fill_missing_gaps(df, not_detect_gaps)
    filled_count = len(filled_df)
    rows_added = filled_count - original_count
    # output file names
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    csv_out = os.path.join(OUTPUT_DIR, f"{base_name}_filled.csv")
    xlsx_out = os.path.join(OUTPUT_DIR, f"{base_name}_filled.xlsx")
    # save
    filled_df.to_csv(csv_out, index=False)
    filled_df.to_excel(xlsx_out, index=False)
    # log entry
    log_entries.append({
        'file': file_path,
        'original_rows': original_count,
        'filled_rows': filled_count,
        'rows_added': rows_added
    })

# Save log summary
log_df = pd.DataFrame(log_entries)
log_path = os.path.join(OUTPUT_DIR, 'filling_summary.csv')
log_df.to_csv(log_path, index=False)
print("Filling completed. Summary:")
print(log_df)


Processing /Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part1.xlsx
Processing /Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part2.xlsx
Processing /Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part3.xlsx
Processing /Users/wss/Desktop/CSV/DJI_0031/step1 single ane and multi lanes analysis_v2_5/combined_trajectories_part4.xlsx
Filling completed. Summary:
                                                file  original_rows  \
0  /Users/wss/Desktop/CSV/DJI_0031/step1 single a...         262143   
1  /Users/wss/Desktop/CSV/DJI_0031/step1 single a...         262143   
2  /Users/wss/Desktop/CSV/DJI_0031/step1 single a...         262053   
3  /Users/wss/Desktop/CSV/DJI_0031/step1 single a...         253935   

   filled_rows  rows_added  
0       265732        3589  
1       265763        3620  
2       265699        3646  
3       